<a href="https://colab.research.google.com/github/profcomff/chatbot-mark-api/blob/dev_fedor/Hitrate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/profcomff/chatbot-mark-api.git

fatal: destination path 'chatbot-mark-api' already exists and is not an empty directory.


In [35]:
CHROMA_DIR = "/content/chatbot-mark-api/chroma_db"

topk = 5
retrivier_k = 5
ensemble_k= 10

grid = [round(x * 0.05, 2) for x in range(21)]
grid = [round(x * 0.1, 2) for x in range(11)]
print(grid)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


# Библиотеки

In [3]:
!pip install langchain transformers sentence-transformers -q
!pip install -U langchain-community -q
!pip install -qU "langchain-chroma>=0.1.2" -q
!pip install langchain_huggingface -q
!pip install rank_bm25 -q

In [4]:
# Core Python libraries
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pickle

from langchain_core.embeddings import Embeddings
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

# Torch and Transformers
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoTokenizer,
    XLMRobertaTokenizer,
    XLMRobertaModel,
)

# NLTK for text preprocessing
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords') #добалено

# Sentence Transformers
from sentence_transformers import CrossEncoder

# LangChain - documents and embeddings
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.vectorstores import InMemoryVectorStore


# LangChain - loading, splitting, and vector stores
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma

from langchain.embeddings import HuggingFaceEmbeddings #this?
from langchain_huggingface import HuggingFaceEmbeddings #or this?

# LangChain - retrieval and reranking
from langchain.retrievers import (
    ContextualCompressionRetriever,
    EnsembleRetriever
)
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.retrievers import BM25Retriever

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import sys
sys.path.append("/content/chatbot-mark-api")

from nn.search import get_context, preprocess #E5LangChainEmbedder,

from sklearn.model_selection import StratifiedShuffleSplit

обновил немного класс, чтобы добавить префиксы

In [15]:
class E5LangChainEmbedder(Embeddings):
    def __init__(
        self,
        tokenizer,
        model,
        device='cpu',
        embed_batch_size=8,
        chroma_batch_size=1000,
        passage_prefix="",
        query_prefix="",
        show_progress=True,
    ):
        self.tokenizer = tokenizer
        self.model = model.to(device)
        self.device = device
        self.embed_batch_size = embed_batch_size
        self.chroma_batch_size = chroma_batch_size
        self.passage_prefix = passage_prefix
        self.query_prefix = query_prefix
        self.show_progress = show_progress
        self.model.eval()

    def _average_pool(self, last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def embed_documents(self, texts, show_progress: bool = None):
        # выбор, показывать ли tqdm
        if show_progress is None:
            show_progress = self.show_progress

        # добавляем prefix к документам, если он есть
        prefixed_texts = [self.passage_prefix + t for t in texts] if self.passage_prefix else texts

        all_embeddings = []
        iterator = range(0, len(prefixed_texts), self.embed_batch_size)
        if show_progress:
            iterator = tqdm(iterator, desc="Вычисление эмбеддингов", unit="batch")

        for i in iterator:
            batch_texts = prefixed_texts[i:i+self.embed_batch_size]
            batch_dict = self.tokenizer(
                batch_texts,
                max_length=512,
                padding=True,
                truncation=True,
                return_tensors='pt'
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**batch_dict)
                embs = self._average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
                embs = torch.nn.functional.normalize(embs, p=2, dim=1)
                all_embeddings.extend(embs.cpu().tolist())

        return all_embeddings

    def embed_query(self, text):
        pref = (self.query_prefix + text) if self.query_prefix else text
        return self.embed_documents([pref], show_progress=False)[0]

# Функции для расчета метрик

In [8]:
def calculate_hitrate_langchain(retriever, questions_df, topk=5, topic_column='topic_name', print_results=True):
    """
    Вычисляет hitrate — долю вопросов, для которых правильный топик есть среди top-k результатов.
    Также выводит подробную информацию по количеству вопросов и попаданий.

    Args:
        retriever: LangChain Retriever с методом get_relevant_documents(query)
        questions_df: pd.DataFrame с колонками 'question' и topic_column (например, 'topik')
        topk: сколько документов брать для подсчёта hitrate
        topic_column: имя колонки с правильным названием топика

    Returns:
        float: hitrate (0..1)
    """
    hits = 0
    total = len(questions_df)

    for _, row in questions_df.iterrows():
        query = row['question']
        correct_topic = row[topic_column]

        docs = retriever.get_relevant_documents(query)[:topk]
        found_topics = [doc.metadata.get('source', '') for doc in docs]

        if correct_topic in found_topics:
            hits += 1

    hitrate = hits / total if total > 0 else 0

    if print_results:
        print(f"Total questions checked: {total}")
        print(f"Correct hits in top-{topk}: {hits}")
        print(f"Hitrate: {hitrate:.3f} ({hitrate*100:.1f}%)")

    return hitrate

In [9]:
def find_best_ensemble_weights(
    bm25_retriever,
    vector_store,
    questions_df,
    topk=5,
    retrivier_k=4,
    ensemble_k=8,
    weights_grid=None,
    topic_column='topic_name',
    verbose=True
):
    """
    Перебирает веса для ансамбля BM25 + vector retriever,
    вычисляет hitrate и возвращает лучшие веса и hitrate.

    Args:
        bm25_retriever: BM25 retriever
        vector_store: vector store (с методом as_retriever)
        questions_df: DataFrame с вопросами и правильными топиками
        topk: сколько документов брать для подсчёта hitrate
        retrivier_k: сколько документов искать в каждом retriever
        ensemble_k: сколько документов возвращает ансамблевый retriever
        weights_grid: список float для перебора веса bm25 (vector вес = 1 - bm25)
        topic_column: имя колонки с правильным топиком
        verbose: выводить ли прогресс

    Returns:
        tuple: (best_weights, best_hitrate)
    """
    if weights_grid is None:
        weights_grid = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]

    vector_retriever = vector_store.as_retriever(search_kwargs={"k": retrivier_k})

    best_hitrate = 0
    best_weights = None

    for bm25_weight in tqdm(weights_grid): #tqdm
        vector_weight = 1.0 - bm25_weight
        weights = [bm25_weight, vector_weight]

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=weights,
            k=ensemble_k
        )

        hitrate = calculate_hitrate_langchain(
            retriever=ensemble_retriever,
            questions_df=questions_df,
            topk=topk,
            topic_column=topic_column
        )

        if verbose:
            print(f"Weights: {weights}, Hitrate: {hitrate:.2%}\n")

        if hitrate > best_hitrate:
            best_hitrate = hitrate
            best_weights = weights

    if verbose:
        print(f"\nBest Weights: {best_weights}, Best Hitrate: {best_hitrate:.2%}")

    return best_weights, best_hitrate

# Загрузка модели и данных


In [16]:
tokenizer = XLMRobertaTokenizer.from_pretrained("d0rj/e5-base-en-ru", use_cache=False)
search_model = XLMRobertaModel.from_pretrained("d0rj/e5-base-en-ru", use_cache=False)

#тоже обновил
embedder = E5LangChainEmbedder(
    tokenizer=tokenizer,
    model=search_model,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    embed_batch_size=4,
    chroma_batch_size=500,
    passage_prefix="passage: ",
    query_prefix="query: "
)

In [17]:
#ручная загрузка бд
if True:
    import zipfile

    archive_path = "/content/bd_chroma_plus_prefix.zip"
    extract_dir = "/content/chroma_db"  # куда распакуем

    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

db_path = "/content/chroma_db/chroma_db" #поправить надо

In [18]:
print(db_path)

vector_store = Chroma(
        collection_name="docs",
        embedding_function=embedder,
        persist_directory=db_path # db_path = "/content/chatbot-mark-api/chroma_db" or CHROMA_DIR
    )

/content/chroma_db/chroma_db


In [19]:
all_docs = vector_store.get(include=["documents", "metadatas"])

documents = [
    Document(page_content=doc_text, metadata=metadata)
    for doc_text, metadata in zip(all_docs["documents"], all_docs["metadatas"])
]

# documents[0:2]

In [20]:
bm25_retriever = BM25Retriever.from_documents(
    documents,
    preprocess_func=preprocess #
)

In [22]:
#debug
model = search_model

results, combined_text = get_context(
    query="как восстановиться после отчисления",
    tokenizer=tokenizer,
    model=model,
    bm_25=bm25_retriever,
    vector_store=vector_store,
    ensemble_k=ensemble_k, #?
    retrivier_k=retrivier_k #?
)

if False:
    print(ensemble_k, retrivier_k)
    display(results)
    display(combined_text)

In [23]:
# questions_deep = pd.read_excel("/content/chatbot-mark-api/file/questions_v2.xlsx")
questions_deep = pd.read_excel("/content/chatbot-mark-api/file/questions_v2_deep.xlsx")
questions_short = pd.read_excel("/content/chatbot-mark-api/file/questions_v2_short.xlsx")

display(questions_deep.head(2))

,topic_name,question,id
0,Карта зачет,Какие преимущества предоставляет профсоюзный б...,0
1,Карта зачет,"Какую информацию можно найти по ссылке, указан...",0


# Расчет метрик на длинных вопросах

In [45]:
#train/test
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

questions = questions_deep
label_column = 'id'

# Получаем индексы train/test
for train_idx, test_idx in splitter.split(questions, questions[label_column]):
    train_df = questions.iloc[train_idx].reset_index(drop=True)
    test_df = questions.iloc[test_idx].reset_index(drop=True)

if True: # Проверка(глупая)
    print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")
    print("Train class distribution:")
    print(train_df[label_column].value_counts(normalize=True).head())
    print("Test class distribution:")
    print(test_df[label_column].value_counts(normalize=True).head())

    display(train_df.head(5))
    display(train_df[train_df.id == 1])
    display(test_df[test_df.id == 1])

Train size: 367, Test size: 158
Train class distribution:
id
56    0.010899
18    0.010899
78    0.010899
96    0.010899
77    0.010899
Name: proportion, dtype: float64
Test class distribution:
id
70    0.012658
54    0.012658
46    0.012658
21    0.012658
71    0.012658
Name: proportion, dtype: float64


,topic_name,question,id
0,Условие перевода на бюджет,Какие шаги необходимо предпринять студенту для...,56
1,Состав КСД,"Какие роли занимают члены комиссии, помимо зам...",48
2,Режим работы и кабинет профкома физфака,"Если у студента есть вопросы в области физики,...",18
3,Выплаты в академическом отпуске,Каковы условия возврата средств за обучение дл...,46
4,Повышенная академическая стипендия,Какие достижения могут стать основанием для по...,78


,topic_name,question,id
102,Как вступить в профсоюз,В какое время можно посетить кабинет Профкома ...,1
226,Как вступить в профсоюз,Почему важно отнести подписанное заявление в к...,1
236,Как вступить в профсоюз,Какие альтернативные способы существуют для по...,1


,topic_name,question,id
103,Как вступить в профсоюз,"Какие шаги нужно предпринять для того, чтобы с...",1
110,Как вступить в профсоюз,Какую роль играет приложение «Твой ФФ» в проце...,1


In [38]:
retriever = bm25_retriever

display(retriever)

hitrate = calculate_hitrate_langchain(retriever,
                                      questions_df=train_df,
                                      topk=topk)

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7bf838491750>, k=5, preprocess_func=<function preprocess at 0x7bf83ab03060>)

Total questions checked: 367
Correct hits in top-5: 337
Hitrate: 0.918 (91.8%)


In [39]:
small_grid = [0.5]
print(grid)

best_weights, best_hitrate = find_best_ensemble_weights(
    bm25_retriever=bm25_retriever,
    vector_store=vector_store,
    questions_df=train_df,
    topk=topk,
    retrivier_k=retrivier_k,
    ensemble_k=ensemble_k,
    weights_grid = grid,
    topic_column='topic_name',
    verbose=True
)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


  0%|          | 0/11 [00:00<?, ?it/s]

Total questions checked: 367
Correct hits in top-5: 311
Hitrate: 0.847 (84.7%)
Weights: [0.0, 1.0], Hitrate: 84.74%

Total questions checked: 367
Correct hits in top-5: 311
Hitrate: 0.847 (84.7%)
Weights: [0.1, 0.9], Hitrate: 84.74%

Total questions checked: 367
Correct hits in top-5: 311
Hitrate: 0.847 (84.7%)
Weights: [0.2, 0.8], Hitrate: 84.74%

Total questions checked: 367
Correct hits in top-5: 311
Hitrate: 0.847 (84.7%)
Weights: [0.3, 0.7], Hitrate: 84.74%

Total questions checked: 367
Correct hits in top-5: 311
Hitrate: 0.847 (84.7%)
Weights: [0.4, 0.6], Hitrate: 84.74%

Total questions checked: 367
Correct hits in top-5: 352
Hitrate: 0.959 (95.9%)
Weights: [0.5, 0.5], Hitrate: 95.91%

Total questions checked: 367
Correct hits in top-5: 337
Hitrate: 0.918 (91.8%)
Weights: [0.6, 0.4], Hitrate: 91.83%

Total questions checked: 367
Correct hits in top-5: 337
Hitrate: 0.918 (91.8%)
Weights: [0.7, 0.30000000000000004], Hitrate: 91.83%

Total questions checked: 367
Correct hits in top

In [40]:
vector_retriever = vector_store.as_retriever(search_kwargs={"k": retrivier_k})

best_ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=best_weights,
    k=ensemble_k
)

# 3. Считаем hitrate на test_df
test_hitrate = calculate_hitrate_langchain(
    retriever=best_ensemble_retriever,
    questions_df=test_df,
    topk=topk,
    topic_column='topic_name'
)

Total questions checked: 158
Correct hits in top-5: 151
Hitrate: 0.956 (95.6%)


# Расчет метрик на коротких вопросах

In [47]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

questions = questions_short
label_column = 'id'

# Получаем индексы train/test
for train_idx, test_idx in splitter.split(questions, questions[label_column]):
    train_df = questions.iloc[train_idx].reset_index(drop=True)
    test_df = questions.iloc[test_idx].reset_index(drop=True)

if True:
    display(train_df.head(5))
    display(train_df[train_df.id == 1])
    display(test_df[test_df.id == 1])

,topic_name,question,id
0,Условие перевода на бюджет,Как часто обновляется информация на сайте?,56
1,Состав КСД,Кто входит в состав КСД?,48
2,Режим работы и кабинет профкома физфака,Где находится физический факультет МГУ?,18
3,Выплаты в академическом отпуске,"Как вернуть деньги за обучение, если в отпуске?",46
4,Повышенная академическая стипендия,Что такое повышенная академическая стипендия?,78


,topic_name,question,id
102,Как вступить в профсоюз,Когда можно прийти в Профком?,1
226,Как вступить в профсоюз,Нужно ли приносить подписанное заявление?,1
236,Как вступить в профсоюз,Что такое «Твой ФФ»?,1


,topic_name,question,id
103,Как вступить в профсоюз,Где находится кабинет Профкома?,1
110,Как вступить в профсоюз,Как подать заявление через приложение?,1


In [48]:
retriever = bm25_retriever

display(retriever)

hitrate = calculate_hitrate_langchain(retriever,
                                      questions_df=train_df,
                                      topk=topk)

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7bf838491750>, k=5, preprocess_func=<function preprocess at 0x7bf83ab03060>)

Total questions checked: 367
Correct hits in top-5: 314
Hitrate: 0.856 (85.6%)


In [49]:
small_grid = [0.5]
print(grid)

best_weights, best_hitrate = find_best_ensemble_weights(
    bm25_retriever=bm25_retriever,
    vector_store=vector_store,
    questions_df=train_df,
    topk=topk,
    retrivier_k=retrivier_k,
    ensemble_k=ensemble_k,
    weights_grid = grid,
    topic_column='topic_name',
    verbose=True
)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


  0%|          | 0/11 [00:00<?, ?it/s]

Total questions checked: 367
Correct hits in top-5: 188
Hitrate: 0.512 (51.2%)
Weights: [0.0, 1.0], Hitrate: 51.23%

Total questions checked: 367
Correct hits in top-5: 188
Hitrate: 0.512 (51.2%)
Weights: [0.1, 0.9], Hitrate: 51.23%

Total questions checked: 367
Correct hits in top-5: 188
Hitrate: 0.512 (51.2%)
Weights: [0.2, 0.8], Hitrate: 51.23%

Total questions checked: 367
Correct hits in top-5: 188
Hitrate: 0.512 (51.2%)
Weights: [0.3, 0.7], Hitrate: 51.23%

Total questions checked: 367
Correct hits in top-5: 188
Hitrate: 0.512 (51.2%)
Weights: [0.4, 0.6], Hitrate: 51.23%

Total questions checked: 367
Correct hits in top-5: 310
Hitrate: 0.845 (84.5%)
Weights: [0.5, 0.5], Hitrate: 84.47%

Total questions checked: 367
Correct hits in top-5: 314
Hitrate: 0.856 (85.6%)
Weights: [0.6, 0.4], Hitrate: 85.56%

Total questions checked: 367
Correct hits in top-5: 314
Hitrate: 0.856 (85.6%)
Weights: [0.7, 0.30000000000000004], Hitrate: 85.56%

Total questions checked: 367
Correct hits in top

In [50]:
vector_retriever = vector_store.as_retriever(search_kwargs={"k": retrivier_k})

best_ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=best_weights,
    k=ensemble_k
)

# 3. Считаем hitrate на test_df
test_hitrate = calculate_hitrate_langchain(
    retriever=best_ensemble_retriever,
    questions_df=test_df,
    topk=topk,
    topic_column='topic_name'
)

Total questions checked: 158
Correct hits in top-5: 135
Hitrate: 0.854 (85.4%)


добавить кросс-энкодер

In [51]:
pass